In [1]:
import pandas as pd
import json
import re
import numpy as np

In [2]:
lake = pd.read_csv("lake_info/lake_information.csv", converters={"lake_id": str})
length = pd.read_csv("fish_length_info/fish_length.csv", converters={"lake_id": str, "survey_id": str})
cpue = pd.read_csv("fish_cpue_info/fish_cpue.csv", converters={"lake_id": str, "survey_id": str})
was = pd.read_csv("was_info/water_access_information.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'lake_info/lake_information.csv'

In [ ]:
#add methods
#sort and format year survey data then convert to datetime
def survey_date_cleaner(dfseries):
    dfseries = dfseries.copy()
    for _ in range(len(dfseries)):
        dfseries[_] = dfseries[_].replace("/","-")
        if matches := re.search(r"^(\d?\d{1})-(\d?\d{1})-(\d{4})$", dfseries[_], re.IGNORECASE):
            dfseries[_] = f"{matches.group(3)}-{matches.group(2)}-{matches.group(1)}"
        try:
            dfseries[_] = pd.to_datetime(dfseries[_],format="%Y-%m-%d")
        except ValueError:
            if matches := re.search(r"^(\d{4})-(\d?\d{1})-(\d?\d{1})$", dfseries[_], re.IGNORECASE):
                dfseries[_] = f"{matches.group(1)}-{matches.group(3)}-{matches.group(2)}"
            dfseries[_] = pd.to_datetime(dfseries[_],format="%Y-%m-%d")
    dfseries = pd.to_datetime(dfseries,format="%Y-%m-%d")
    return dfseries

#input CPUE column data series
#drop strings from cpue data series and convert to int(found some "? marks")
def cpue_cleaner(dfseries):
    i=0
    for _ in dfseries:
        try:
            np.float64(_)
            i = i + 1
        except ValueError:
            print(i, _,"valueerror")
            dfseries = dfseries.drop([i])
            i = i + 1
    dfseries = pd.to_numeric(dfseries)
    return dfseries

#input fish_count column data series
#returns fish_count column in dataframe as a list of individual integer measurements e.g. [0,0,1,1,1,2,3,3,4]
def fish_length_cleaner(dfseries):
    fish_lengths = []
    #scrub each row string and convert to list of lengths sampled
    for row in dfseries:
        #clean up string of list syntax
        sample = row.replace("]","").replace("[","").replace(" ","").split(",")
        #breakdown string into list of lengths sampled for row
        sample_lengths = []
        for n in range(len(sample)):
            if n % 2 == 1:
                fishlength = int(sample[n-1])
                fishcount = int(sample[n])
                for c in range(fishcount):
                    sample_lengths.append(fishlength)
        fish_lengths.append(sample_lengths)
    return fish_lengths

#input string data series with rows of "[1,1,2,2,2,3,4,7,7,7]"
#returns fish_count series column as a list of float averages e.g. [5.1, 4.6, 8.8]
def fish_length_averager(dfseries):
    fish_length_averages = []
    for row in dfseries:
        sample = [int(x) for x in row if x != 0]
        if bool(sample) == True:
            fish_length_averages.append(np.mean(sample))
        else:
            fish_length_averages.append(np.nan)
    return fish_length_averages

#input quartile data series
#split lower quartile and convert to float
def lower_quartile_cleaner(dseries):
    quartile_list = []
    for quartile in dseries:
        try:
            if len(quartile.split("-")) == 2:
                try:
                    quartile_list.append(float(quartile.split("-")[0]))
                except ValueError:
                    quartile_list.append(float("nan"))
                    pass
            else:
                quartile_list.append(float("nan"))
        except AttributeError:
            quartile_list.append(quartile)
            pass
    return quartile_list

#input quartile data series
#split upper quartile and convert to float
def upper_quartile_cleaner(dseries):
    quartile_list = []
    for quartile in dseries:
        try:
            if len(quartile.split("-")) == 2:
                try:
                    quartile_list.append(float(quartile.split("-")[1]))
                except ValueError:
                    quartile_list.append(float("nan"))
                    pass
            else:
                quartile_list.append(float("nan"))
        except AttributeError:
            quartile_list.append(quartile)
            pass
    return quartile_list



In [ ]:
lake.info()

In [3]:
#add lat and lon columns and drop coordinates column
lake_df = lake.drop_duplicates()
lake_coordinates = [json.loads(x) for x in lake_df["lake_coordinates"]]
longitude = [x[0] for x in lake_coordinates]
latitude = [x[1] for x in lake_coordinates]
lake_df["lake_longitude"] = longitude
lake_df["lake_latitude"] = latitude
lake_df = lake_df.drop(columns=["lake_coordinates"])
lake_df.head()

NameError: name 'lake' is not defined

In [4]:
lake_df.info()

NameError: name 'lake_df' is not defined

In [5]:
was.info()

NameError: name 'was' is not defined

In [ ]:
#add lat lon columns and drop coordinates column
was_df = was.drop_duplicates()
site_coordinates = [json.loads(x) for x in was_df["was_site_coordinates"]]
longitude = [x[0] for x in site_coordinates]
latitude = [x[1] for x in site_coordinates]
was_df["was_site_longitude"] = longitude
was_df["was_site_latitude"] = latitude
was_df = was_df.drop(columns=["was_site_coordinates"])
was_df.head()

In [ ]:
was_df.info()

In [ ]:
length.info()

In [ ]:
#clean up the length file duplicates, clean dates, convert lengths in count to histogram, and create average column
length_df = length.drop_duplicates()
length_df["survey_date"] = survey_date_cleaner(length_df["survey_date"])
length_df["fish_count"] = fish_length_cleaner(length_df["fish_count"])
length_df["average_length"] = fish_length_averager(length_df["fish_count"])
length_df = length_df.sort_values("survey_date")
#add length_id column
length_df = length_df.reset_index(drop=True)
length_id = length_df.index
length_df.insert(loc=0, column="length_id", value=length_id)
length_df.head()

In [ ]:
length_df.info()

In [ ]:
cpue.info()

In [ ]:
#clean cpue and date and separate quartiles into columns.
cpue_df = cpue.drop_duplicates()
cpue_df["cpue"] = cpue_cleaner(cpue_df["cpue"])
cpue_df["survey_date"] = survey_date_cleaner(cpue_df["survey_date"])
cpue_df["count_lower_quartile"] = lower_quartile_cleaner(cpue_df["quartile_count"])
cpue_df["count_upper_quartile"] = upper_quartile_cleaner(cpue_df["quartile_count"])
cpue_df["weight_lower_quartile"] = lower_quartile_cleaner(cpue_df["quartile_weight"])
cpue_df["weight_upper_quartile"] = upper_quartile_cleaner(cpue_df["quartile_weight"])
cpue_df["gear_count"] = cpue["gear_count"].astype("int")
cpue_df = cpue_df.drop(columns=["quartile_count", "quartile_weight"])
cpue_df = cpue_df.sort_values("survey_date")
#add cpue_id column
cpue_df = cpue_df.reset_index(drop=True)
cpue_id = cpue_df.index
cpue_df.insert(loc=0, column="cpue_id", value=cpue_id)
cpue_df.head()

In [ ]:
cpue_df.info()

In [ ]:
#create csv files to load into database
lake_df.to_csv("lake_info/lake_information_clean.csv", index=False)
length_df.to_csv("fish_length_info/fish_length_clean.csv", index=False)
cpue_df.to_csv("fish_cpue_info/fish_cpue_clean.csv", index=False)

In [ ]:
#add lake id to water access sites
was_values = [x.replace("\'","\"") for x in list(lake_df["water_access_sites"])]
was_values = [json.loads(x) for x in was_values]
lake_df["was_site_id"] = was_values
was_lake = lake_df.loc[lake_df["was_site_id"].str.len() > 0, ["lake_id", "lake_name", "was_site_id"]]
was_exploded = was_lake.explode("was_site_id")
was_exploded_clean = was_exploded.drop_duplicates(subset = "was_site_id")
was_df = was_df.merge(was_exploded_clean, on="was_site_id", how="left")
was_df.head(5)

In [ ]:
#create csv file to load into database
was_df.to_csv("was_info/water_access_information_clean.csv", index=False)